**352 ROB -- Optmized MC**

**4-core, MOP, Open-Page, RFMsb w/o Early Reset Results (PrIDE/Mint/Mithril)**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/rfm_results/optimized_MC/results_4cores'

df = pd.DataFrame(columns=["mitigation", "workload"])
df_baseline = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["RFMsb", "RFMpb", "Baseline"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        workload = result_filename

        if mitigation not in ['Baseline']:
            BAT = int(result_filename.split("_")[0])
            workload = "_".join(result_filename.split("_")[1:])


        # workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3

        result_file.close()
        # Create a new DataFrame for the new row
        if mitigation in ["Baseline"]:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df_baseline = pd.concat([df_baseline, new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'BAT': [BAT],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df = pd.concat([df, new_row], ignore_index=True)

df_sc_ipc = pd.read_csv('../stats/SC_500M_OPTMC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel']==1) & (df_sc_ipc['interface'] == 6400)]
df_sc_ipc = df_sc_ipc.drop(columns=['Channel', 'interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df_baseline['normalzied_ipc0'] = df_baseline['ipc0'] / df_baseline['ipc_wl0']
df_baseline['normalzied_ipc1'] = df_baseline['ipc1'] / df_baseline['ipc_wl1']
df_baseline['normalzied_ipc2'] = df_baseline['ipc2'] / df_baseline['ipc_wl2']
df_baseline['normalzied_ipc3'] = df_baseline['ipc3'] / df_baseline['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)
df_baseline['WS'] = df_baseline[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_ws_openpage = df[['mitigation', 'BAT', 'workload','WS']]
df_baseline_ws_openpage = df_baseline[['mitigation', 'workload','WS']]
# print(df_baseline_ws_openpage)
# print(df_ws_openpage)

# Merge df_ws with df_baseline_ws on the 'workload' column
merged_df = pd.merge(df_ws_openpage, df_baseline_ws_openpage[['workload', 'WS']], on='workload', how='left', suffixes=('', '_baseline'))
# Calculate normalized WS
merged_df['normalized_WS'] = merged_df['WS'] / merged_df['WS_baseline']

# Display the result
# print(merged_df[['mitigation', 'BAT', 'workload', 'normalized_WS']])
merged_df_pivot = merged_df.pivot(index=['workload', 'BAT'], columns=['mitigation'], values='normalized_WS').reset_index()

# print(merged_df_pivot)

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 22
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    # 'LonestarGPU (3)': ['bfs_dblp', 'bfsbfs_cm2003_cm_2003','bfs_ny'], #lnestargpu: 3
    # 'MediaBench (4)': ['h264_decode', 'h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 4
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for BAT in df['BAT'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['BAT'] == BAT)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'BAT': BAT, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for BAT in df['BAT'].unique():
        Channel_interface_df = df[(df['BAT'] == BAT)]
        geomean_values = {}

        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'BAT': BAT, 'workload': 'All (57)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Call function to calculate and merge geometric means
mitigation_list = ['RFMsb', 'RFMpb']
geomean_rfm_reset_ws = add_geomean_rows(merged_df_pivot)
geomean_rfm_reset_ws = add_all_workloads_geomean_rows(geomean_rfm_reset_ws)
print(geomean_rfm_reset_ws)
geomean_rfm_reset_ws.to_csv('../stats/RFMsb_NoReset_500M_4homogeneous_OptimizedMC.csv', index=False)
# geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
# geomean_df = add_all_workloads_geomean_rows(geomean_df)
# print(geomean_df)
# geomean_df.to_csv('./stats/RFM_500M_4homogeneous.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: './stats/SC_500M_OPTMC_Baseline.csv'

**4-core, MOP, Open-Page, RFM w/ Early Reset Results**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../rfm_results/optimized_MC/results_4cores'

df = pd.DataFrame(columns=["mitigation", "workload"])
df_baseline = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["RFMab-Reset", "RFMsb-Reset", "Baseline"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        workload = result_filename

        if mitigation not in ['Baseline']:
            BAT = int(result_filename.split("_")[0])
            workload = "_".join(result_filename.split("_")[1:])


        # workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3

        result_file.close()
        # Create a new DataFrame for the new row
        if mitigation in ["Baseline"]:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df_baseline = pd.concat([df_baseline, new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'BAT': [BAT],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df = pd.concat([df, new_row], ignore_index=True)

# print(df_baseline)
df_sc_ipc = pd.read_csv('./stats/SC_500M_OPTMC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel']==1) & (df_sc_ipc['interface'] == 6400)]
df_sc_ipc = df_sc_ipc.drop(columns=['Channel', 'interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df_baseline['normalzied_ipc0'] = df_baseline['ipc0'] / df_baseline['ipc_wl0']
df_baseline['normalzied_ipc1'] = df_baseline['ipc1'] / df_baseline['ipc_wl1']
df_baseline['normalzied_ipc2'] = df_baseline['ipc2'] / df_baseline['ipc_wl2']
df_baseline['normalzied_ipc3'] = df_baseline['ipc3'] / df_baseline['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)
df_baseline['WS'] = df_baseline[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_ws_openpage = df[['mitigation', 'BAT', 'workload','WS']]
df_baseline_ws_openpage = df_baseline[['mitigation', 'workload','WS']]
# print(df_baseline_ws_openpage)
# print(df_ws_openpage)

# Merge df_ws with df_baseline_ws on the 'workload' column
merged_df = pd.merge(df_ws_openpage, df_baseline_ws_openpage[['workload', 'WS']], on='workload', how='left', suffixes=('', '_baseline'))
# Calculate normalized WS
merged_df['normalized_WS'] = merged_df['WS'] / merged_df['WS_baseline']

# Display the result
# print(merged_df[['mitigation', 'BAT', 'workload', 'normalized_WS']])
merged_df_pivot = merged_df.pivot(index=['workload', 'BAT'], columns=['mitigation'], values='normalized_WS').reset_index()

# print(merged_df_pivot)

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 22
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    # 'LonestarGPU (3)': ['bfs_dblp', 'bfsbfs_cm2003_cm_2003','bfs_ny'], #lnestargpu: 3
    # 'MediaBench (4)': ['h264_decode', 'h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 4
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for BAT in df['BAT'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['BAT'] == BAT)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'BAT': BAT, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for BAT in df['BAT'].unique():
        Channel_interface_df = df[(df['BAT'] == BAT)]
        geomean_values = {}

        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'BAT': BAT, 'workload': 'All (57)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Call function to calculate and merge geometric means
mitigation_list = ['RFMab-Reset', 'RFMsb-Reset']
geomean_rfm_reset_ws = add_geomean_rows(merged_df_pivot)
geomean_rfm_reset_ws = add_all_workloads_geomean_rows(geomean_rfm_reset_ws)
print(geomean_rfm_reset_ws)
geomean_rfm_reset_ws.to_csv('./stats/RFM_Reset_500M_4homogeneous_OptimizedMC.csv', index=False)
# geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
# geomean_df = add_all_workloads_geomean_rows(geomean_df)
# print(geomean_df)
# geomean_df.to_csv('./stats/RFM_500M_4homogeneous.csv', index=False)


      workload    BAT  RFMab-Reset  RFMsb-Reset
0    401.bzip2   40.0     0.980184     0.995539
1    401.bzip2   70.0     0.997686     1.003284
2    401.bzip2  100.0     0.993785     0.999167
3    401.bzip2  140.0     0.992330     1.007254
4      403.gcc   40.0     0.997925     0.999310
..         ...    ...          ...          ...
251   YCSB (6)  140.0     0.993203     0.996724
252   All (57)   40.0     0.958309     0.978668
253   All (57)   70.0     0.977934     0.988346
254   All (57)  100.0     0.985079     0.991961
255   All (57)  140.0     0.989491     0.994372

[256 rows x 4 columns]


**352 ROB -- Non Optmized MC**

**4-Cores, RoBaRaCoCh, Open-Page, RFMsb w/o Early Reset Results (PrIDE/Mint/Mithril)**

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../rfm_results/results_4cores'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["RFMsb"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        BAT = int(result_filename.split("_")[0])

        workload = "_".join(result_filename.split("_")[1:])
        # workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'BAT': [BAT],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# print(df)
df_sc_ipc = pd.read_csv('./stats/SC_500M_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel']==1) & (df_sc_ipc['interface'] == 6400)]
df_sc_ipc = df_sc_ipc.drop(columns=['Channel', 'interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_rfm_methods_ws_openpage = df[['mitigation', 'BAT', 'workload','WS']]
df_rfm_methods_ws_open_pivot = df_rfm_methods_ws_openpage.pivot(index=['workload', 'BAT'], columns=['mitigation'], values='WS').reset_index()
print(df_rfm_methods_ws_open_pivot)
# for mitigation in set(mitigation_list) - set(['Baseline']):
#      df_ws_openpage_pivot[mitigation] = df_ws_openpage_pivot[mitigation] / df_ws_openpage_pivot['Baseline']
# df_ws_openpage_pivot.drop(columns=['Baseline'], inplace=True)

# print(df_ws_openpage)
# ##### Calculate the Geomean for each workload type
# # Define benchmark suites and their corresponding workloads ranges
# benchmark_types = {
#     'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
#     'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
#     'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
#     'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
#     'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
#     'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
# }

# # DataFrame Example (you already have your df)
# # Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# # Add a new column to assign each Mix to the appropriate suite
# def assign_benchmark_types(mix_index):
#     for suite, mix_range in benchmark_types.items():
#         if mix_index in mix_range:
#             return suite
#     return None

# # Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
# df_closed_cap1_ws_pivot['Mix_index'] = df_closed_cap1_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
# df_closed_cap1_ws_pivot['Benchmark_Types'] = df_closed_cap1_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# # Function to calculate the geometric mean
# def geom_mean(series):
#     return np.exp(np.log(series).mean())

# # Function to calculate and add geometric means as new rows
# def add_geomean_rows(df):
#     geomean_rows = []  # List to collect new rows

#     for Channel in df['Channel'].unique():
#         for suite_name, mix_indices in benchmark_types.items():
#             # Create a list of corresponding workload names (e.g., MIX0, MIX1)
#             workloads = [f'MIX{i}' for i in mix_indices]
#             suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel)]
#             if not suite_df.empty:
#                 geomeans = {}
                
#                 # Dynamically calculate geometric means for each mitigation
#                 for mitigation in mitigation_list:
#                     geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
#                 # Create a new row
#                 geomean_row = {'Channel': Channel, 'workload': suite_name, **geomeans}
#                 geomean_rows.append(geomean_row)  # Append to the list

#     # Convert list of rows to DataFrame
#     geomean_df = pd.DataFrame(geomean_rows)
    
#     return pd.concat([df, geomean_df], ignore_index=True)

# def add_all_workloads_geomean_rows(df):
#     geomean_rows = []  # List to collect new rows
    
#     for Channel in df['Channel'].unique():
#         Channel_df = df[df['Channel'] == Channel]
#         geomean_values = {}
        
#         # Calculate geometric means for each mitigation in the list
#         for mitigation in mitigation_list:
#             if mitigation in Channel_df.columns:  # Ensure the column exists
#                 geomean_values[mitigation] = calculate_geometric_mean(Channel_df[mitigation])

#         # Create a new row for the combined results
#         geomean_row = {'Channel': Channel, 'workload': 'All (90)', **geomean_values}
#         geomean_rows.append(geomean_row)  # Append to the list
    
#     # Convert list of rows to DataFrame
#     geomean_df = pd.DataFrame(geomean_rows)
    
#     return pd.concat([df, geomean_df], ignore_index=True)

# mitigation_list = ["PRAC_WO_Mitigation-ClosedCap1"]


# geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
# geomean_df = add_all_workloads_geomean_rows(geomean_df)
# print(geomean_df)
# geomean_df.to_csv('./stats/RFM_500M_4homogeneous.csv', index=False)


mitigation      workload   BAT     RFMsb
0              401.bzip2   1.0  0.312663
1              401.bzip2   2.0  0.790512
2              401.bzip2   5.0  1.579479
3              401.bzip2  10.0  2.081050
4              401.bzip2  40.0  2.705606
..                   ...   ...       ...
333         ycsb_eserver   2.0  0.416433
334         ycsb_eserver   5.0  0.991393
335         ycsb_eserver  10.0  1.540306
336         ycsb_eserver  40.0  2.392663
337         ycsb_eserver  70.0  2.575094

[338 rows x 3 columns]


**4-Cores, MOP Mapping, RFM with Early Reset Results OpenRowPolicy**

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../rfm_results/results_4cores'

df = pd.DataFrame(columns=["mitigation", "workload"])
df_baseline = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["RFMab-Reset", "RFMsb-Reset", "Baseline"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        if mitigation not in ['Baseline']:
            BAT = int(result_filename.split("_")[0])

        workload = "_".join(result_filename.split("_")[1:])
        # workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3

        result_file.close()
        # Create a new DataFrame for the new row
        if mitigation in ["Baseline"]:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df_baseline = pd.concat([df_baseline, new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame({
                'mitigation': [mitigation],
                'workload': [workload],
                'BAT': [BAT],
                'wl0': [w0],
                'wl1': [w1],
                'wl2': [w2],
                'wl3': [w3],
                'ipc0': [ipc_0],
                'ipc1': [ipc_1],
                'ipc2': [ipc_2],
                'ipc3': [ipc_3],
            })
            df = pd.concat([df, new_row], ignore_index=True)

df_sc_ipc = pd.read_csv('./stats/SC_500M_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel']==1) & (df_sc_ipc['interface'] == 6400)]
df_sc_ipc = df_sc_ipc.drop(columns=['Channel', 'interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl0'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl1'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl2'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
df_baseline = df_baseline.merge(df_sc_ipc[['workload_sc', 'Baseline']], left_on=['wl3'], right_on=['workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup
    df_baseline = df_baseline.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df_baseline['normalzied_ipc0'] = df_baseline['ipc0'] / df_baseline['ipc_wl0']
df_baseline['normalzied_ipc1'] = df_baseline['ipc1'] / df_baseline['ipc_wl1']
df_baseline['normalzied_ipc2'] = df_baseline['ipc2'] / df_baseline['ipc_wl2']
df_baseline['normalzied_ipc3'] = df_baseline['ipc3'] / df_baseline['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)
df_baseline['WS'] = df_baseline[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_ws_openpage = df[['mitigation', 'BAT', 'workload','WS']]
df_baseline_ws_openpage = df_baseline[['mitigation', 'workload','WS']]
# print(df_baseline_ws_openpage)
# print(df_ws_openpage)

# Merge df_ws with df_baseline_ws on the 'workload' column
merged_df = pd.merge(df_ws_openpage, df_baseline_ws_openpage[['workload', 'WS']], on='workload', how='left', suffixes=('', '_baseline'))
# Calculate normalized WS
merged_df['normalized_WS'] = merged_df['WS'] / merged_df['WS_baseline']

# Display the result
# print(merged_df[['mitigation', 'BAT', 'workload', 'normalized_WS']])
merged_df_pivot = merged_df.pivot(index=['workload', 'BAT'], columns=['mitigation'], values='normalized_WS').reset_index()

# print(merged_df_pivot)

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 22
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    # 'LonestarGPU (3)': ['bfs_dblp', 'bfsbfs_cm2003_cm_2003','bfs_ny'], #lnestargpu: 3
    # 'MediaBench (4)': ['h264_decode', 'h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 4
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for BAT in df['BAT'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['BAT'] == BAT)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'BAT': BAT, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for BAT in df['BAT'].unique():
        Channel_interface_df = df[(df['BAT'] == BAT)]
        geomean_values = {}

        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'BAT': BAT, 'workload': 'All (57)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Call function to calculate and merge geometric means
mitigation_list = ['RFMab-Reset', 'RFMsb-Reset']
geomean_rfm_reset_ws = add_geomean_rows(merged_df_pivot)
geomean_rfm_reset_ws = add_all_workloads_geomean_rows(geomean_rfm_reset_ws)
print(geomean_rfm_reset_ws)
geomean_rfm_reset_ws.to_csv('./stats/RFM_Reset_500M_4homogeneous.csv', index=False)
# geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
# geomean_df = add_all_workloads_geomean_rows(geomean_df)
# print(geomean_df)
# geomean_df.to_csv('./stats/RFM_500M_4homogeneous.csv', index=False)


      workload    BAT  RFMab-Reset  RFMsb-Reset
0    401.bzip2   40.0     0.986082     0.973553
1    401.bzip2   70.0     0.988595     0.981370
2    401.bzip2  100.0     0.993958     0.985553
3    401.bzip2  140.0     0.986823     0.987618
4      403.gcc   40.0     0.999423     0.998078
..         ...    ...          ...          ...
251   YCSB (6)  140.0     0.996113     0.993336
252   All (57)   40.0     0.973328     0.951822
253   All (57)   70.0     0.987317     0.973465
254   All (57)  100.0     0.991135     0.982107
255   All (57)  140.0     0.993769     0.987487

[256 rows x 4 columns]
